In [1]:
from google.colab import files
uploaded = files.upload()

Saving train (1).csv to train (1).csv


In [83]:
import pandas  as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [68]:
raw_data = pd.read_csv("train (1).csv")

In [69]:
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [71]:
raw_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [72]:
raw_data.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [73]:
raw_data.duplicated().sum()

np.int64(0)

In [74]:
df = raw_data.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [75]:
raw_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [76]:
df = raw_data.copy()

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns= ['Survived']),
                                                df['Survived'],
                                                test_size=0.2,
                                                random_state=42)

In [79]:
y_test.head()

,Survived
709,1
439,0
840,0
720,1
39,1


In [95]:
## Impute Transfromer

impute_transfromer = ColumnTransformer(
    transformers = [
        ('impute_age',SimpleImputer(),[2]),
        ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
    ],
    remainder = 'passthrough'
)

In [102]:
## OneHotEncoder
one_transformer = ColumnTransformer(
    transformers= [
        ('one_hot',OneHotEncoder(handle_unknown= 'ignore', sparse_output  = False), [1,3]) # Corrected indices to include Sex (index 3)
    ] ,
    remainder = 'passthrough'
)

In [103]:
## Normalization
Normalization_transformer = ColumnTransformer(
    transformers = [
        ('scale',MinMaxScaler(),slice(0,10))
    ],
    remainder = 'passthrough'
)

In [104]:
from sklearn.feature_selection import SelectKBest,chi2
## Feature Slectioon
feature_selection_transformer  = SelectKBest(score_func=chi2,k= 5 )

In [105]:
## Model
Model = DecisionTreeClassifier()

## **Create pipeline**

In [106]:

pipe = Pipeline([
    ('impute_transfromer',impute_transfromer),
    ('one_transformer',one_transformer),
    ('Normalization_transformer',Normalization_transformer),
    ('feature_selection_transformer',feature_selection_transformer),
    ('Model',Model)
])

## Train

In [107]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('impute_transfromer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('one_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('Normalization_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('feature_selection_transformer',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7e91bd293c40>)),
                ('Model', DecisionTreeClassifier())])

In [113]:
pipe.named_steps['impute_transfromer'].transformers_[0][1].statistics_

array([29.49884615])

In [115]:
y_pred = pipe.predict(X_test)

In [116]:
accuracy_score(y_test,y_pred)

0.8044692737430168

In [117]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv = 5 ,scoring ='accuracy').mean()

np.float64(0.789421845759874)

## **GridSerach Using Pipeline**

In [123]:
params = {
          'Model__max_depth' : [2,4,6,8,10,12,None],
}

In [124]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv = 5,scoring = 'accuracy')

In [125]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('impute_transfromer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('one_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one_hot',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sp...=False),
                                                                         [1,
                                                                          3])])),
                                       ('Normalization_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('feature_selection_transformer',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x7e91bd293c40>)),
                                       ('Model', DecisionTreeClassifier())]),
             param_grid={'Model__max_depth': [2, 4, 6, 8, 10, 12, None]},
             scoring='accuracy')

In [126]:
grid.best_score_

np.float64(0.8090121146459175)

In [127]:
grid.best_params_

{'Model__max_depth': 8}

# **Exporting The Pipeline**




In [128]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))